In [67]:
import pandas as pd
import neo4j
import math

In [68]:
# Created neo4j desktop dbms with password "senate"
driver = neo4j.GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "senate"))

# Got import dir from neo4j desktop browser
neo4j_import_dir = "/Users/aidankeogh/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-dffef9db-3d5a-4ba6-aebc-aadc8b8f5988/import"

In [69]:

def run(query):
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            print(record)

In [70]:
run("""
MATCH (n)
DETACH DELETE n
""")

In [71]:
senators = pd.read_csv("S117_members.csv")

code_to_party = {100: "democrat", 200: "republican"}
def create_senator(senator):
    name = senator["bioname"]
    node_attrs = {
        "lastname": name.split(",")[0].lower(),
        "firstname": name.split(",")[1].split()[0].lower(),
        "party": code_to_party.get(senator["party_code"], "other"),
        "state": senator["state_abbrev"],
        "icpsr": senator["icpsr"]
    }
    return pd.Series(node_attrs)
senator_nodes = senators.apply(create_senator, axis=1)
senator_nodes.to_csv(f"{neo4j_import_dir}/senator_nodes.csv")

In [72]:

run("""
LOAD CSV WITH HEADERS FROM "file:///senator_nodes.csv" as row
MERGE (s:Senator {lastname: row.lastname, firstname: row.firstname, party:row.party, state: row.state, id: toInteger(row.icpsr)})
""")

In [73]:
run("""
MATCH (s: Senator {party: "other"})
RETURN s.firstname, s.lastname, s.party, s.id
""")

<Record s.firstname='angus' s.lastname='king' s.party='other' s.id=41300>
<Record s.firstname='bernard' s.lastname='sanders' s.party='other' s.id=29147>


In [74]:
rollcalls

,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question,dtl_desc
0,117,Senate,1,2021-01-06,1,1,6,93,-0.695,0.620,0.115,-1.472,-18.659,NaN,Objection Not Sustained,NaN,On the Objection,NaN
1,117,Senate,2,2021-01-07,1,2,7,92,-0.767,0.609,0.119,-1.265,-19.668,NaN,Objection Not Sustained,NaN,On the Objection,NaN
2,117,Senate,3,2021-01-20,1,3,84,10,0.541,0.841,0.450,0.340,-14.842,PN7810,Nomination Confirmed,"Avril Danica Haines, of New York, to be Direct...",On the Nomination,NaN
3,117,Senate,4,2021-01-21,1,4,69,27,-0.787,0.617,-0.132,-0.259,-51.575,HR335,Bill Passed,A bill to provide for an exception to a limita...,On Passage of the Bill,NaN
4,117,Senate,5,2021-01-22,1,5,93,2,0.000,0.000,0.000,0.000,0.000,PN781,Nomination Confirmed,"Lloyd James Austin, of Georgia, to be Secretar...",On the Nomination,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,117,Senate,852,2022-08-07,2,324,50,50,0.042,-0.077,0.677,-0.559,-0.142,HR5376,Amendment Agreed to,To strike the extension of the limitation on S...,On the Amendment,NaN
852,117,Senate,853,2022-08-07,2,325,50,50,0.042,-0.077,0.677,-0.559,-0.142,HR5376,Bill Passed,A bill to provide for reconciliation pursuant ...,On Passage of the Bill,NaN
853,117,Senate,854,2022-09-06,2,326,48,42,-0.112,0.935,0.508,0.998,-7.606,PN1965,Cloture Motion Agreed to,"John Z. Lee, of Illinois, to be United States ...",On the Cloture Motion,NaN
854,117,Senate,855,2022-09-07,2,327,50,44,-0.111,0.926,0.509,1.007,-7.614,PN1965,Nomination Confirmed,"John Z. Lee, of Illinois, to be United States ...",On the Nomination,NaN


In [75]:

rollcalls = pd.read_csv("S117_rollcalls.csv")

def create_rollcall(rollcall):
    node_attrs = {
        "rollnumber": rollcall["rollnumber"],
        "date": rollcall["date"],
        "yea_count": rollcall["yea_count"],
        "nea_count": rollcall["nay_count"],
        "result": rollcall["vote_result"],
        "desc": rollcall["vote_desc"],
        "bill_number": rollcall["bill_number"],
    }
    return pd.Series(node_attrs)

rollcall_nodes = rollcalls.apply(create_rollcall, axis=1)
rollcall_nodes["bill_number"] = rollcall_nodes["bill_number"].fillna("N/A")
rollcall_nodes.to_csv(f"{neo4j_import_dir}/rollcall_nodes.csv")

In [76]:

run("""
LOAD CSV WITH HEADERS FROM "file:///rollcall_nodes.csv" as row
WITH row WHERE row.desc is not null
MERGE (r:Rollcall {rollnumber: toInteger(row.rollnumber), date: row.date, yea_count: toInteger(row.yea_count), 
    nea_count: toInteger(row.nea_count), result: row.result, desc: row.desc, bill_number: row.bill_number})
""")

In [77]:
run("""
MATCH (r: Rollcall)
WHERE r.yea_count > 90
RETURN r.rollnumber, r.desc, r.yea_count
ORDER by r.yea_count DESC
""")

<Record r.rollnumber=16 r.desc='To establish a deficit-neutral reserve fund relating to prohibiting legislation that would increase taxes on small businesses during any period in which a national emergency has been declared with respect to a pandemic.' r.yea_count=100>
<Record r.rollnumber=21 r.desc='To establish a deficit-neutral reserve fund relating to COVID-19 vaccine administration and a public awareness campaign.' r.yea_count=100>
<Record r.rollnumber=27 r.desc='To establish a deficit-neutral reserve fund relating to improving services and interventions relating to sexual assault, family violence, domestic violence, dating violence, and child abuse.' r.yea_count=100>
<Record r.rollnumber=35 r.desc='To establish a deficit-neutral fund relating to funding the police.' r.yea_count=100>
<Record r.rollnumber=272 r.desc='A bill to deposit certain funds into the Crime Victims Fund, to waive matching requirements, and for other purposes.' r.yea_count=100>
<Record r.rollnumber=659 r.desc=

In [78]:
votes = pd.read_csv("S117_votes.csv")
vote_types = {
    1: "yea",
    6: "nay",
}
def create_vote_rel(vote):
    icpsr = vote["icpsr"]
    rollnumber = vote["rollnumber"]
    vote_type = vote_types.get(vote["cast_code"], "other")

    return pd.Series({
        "senator_id": int(icpsr),
        "rollnumber": int(rollnumber),
        "vote_type": vote_type
    })
vote_rels = votes.apply(create_vote_rel, axis=1)
vote_rels.to_csv(f"{neo4j_import_dir}/vote_relationships.csv")


In [79]:
run("""
LOAD CSV WITH HEADERS FROM "file:///vote_relationships.csv" as row
MATCH
  (s:Senator {id: toInteger(row.senator_id)}),
  (r:Rollcall {rollnumber: toInteger(row.rollnumber)})
MERGE (s)-[v:Voted {vote_type: row.vote_type}]->(r)
""")

In [83]:
run("""
MATCH (s:Senator {firstname: "bernard", lastname: "sanders"})-[v]-(r)
RETURN v.vote_type, r.desc, r.bill_number
""")

<Record v.vote_type='nay' r.desc='A joint resolution providing for congressional disapproval under chapter 8 of title 5, United States Code, of the rule submitted by the Centers for Medicare &#38; Medicaid Services relating to "Medicare and Medicaid Programs; Omnibus COVID-19 Health Care Staff Vaccination".' r.bill_number='SJRES32'>
<Record v.vote_type='other' r.desc='David Augustin Ruiz, of Ohio, to be United States District Judge for the Northern District of Ohio' r.bill_number='PN1506'>
<Record v.vote_type='yea' r.desc='To provide coverage for dental and oral health care, hearing care, and vision care under the Medicare program.' r.bill_number='HR5376'>
<Record v.vote_type='yea' r.desc='Dara Lindenbaum, of Virginia, to be a Member of the Federal Election Commission for a term expiring April 30, 2027' r.bill_number='PN1758'>
<Record v.vote_type='yea' r.desc='Sarah Bianchi, of Virginia, to be Deputy United States Trade Representative (Asia, Africa, Investment, Services, Textiles, and 